## Here is panos code
We need to create 3 tests with brute force search(O(n))
1) with Linf metric
2) with Linf + Clustering
3) with Linf complete
4) with L2 metric

In [3]:
import numpy as np

In [13]:
# define Linf() function for a square and a point
def Linf(square, point):
    # 1st put coords around point and transform
    # in order to do the transform substract the point from each coord
    square_new = np.array([np.subtract(square_point, point) for square_point in square])
    # find if y = abs(x) intersects the square and if so add the point to the list so that the clockwise order is preserved
    # find the intersection point
    # find if y = abs(x) intersects the square
    # for each edge of the square, check if y = abs(x) intersects the edge
    for i in range(square.shape[0]):
        sq_pt1 = square_new[i]
        sq_pt2 = square_new[(i+1)%square.shape[0]]
        # alpha 
        if not (sq_pt2[0] - sq_pt1[0] == 0):
            a = (sq_pt2[1] - sq_pt1[1])/(sq_pt2[0] - sq_pt1[0])
        else:
            continue
        # beta
        b = sq_pt1[1] - a*sq_pt1[0]
        # new point
        if not (a == 1 or a == -1):
            new_pts = np.array([[b/(a-1), b/(a-1)],[b/(a+1), -b/(a+1)]])
        else:
            continue
        # check if new points lie inside the segment
        # if so add to square_new
        for new_pt in new_pts:
            if (new_pt[0] >= min(sq_pt1[0], sq_pt2[0])) and (new_pt[0] <= max(sq_pt1[0], sq_pt2[0])) and \
                (new_pt[1] >= min(sq_pt1[1], sq_pt2[1])) and (new_pt[1] <= max(sq_pt1[1], sq_pt2[1])):
                square_new = np.insert(square_new, i+1, new_pt, axis=0)
    # get Linf from each point in square_new
    min_dist = np.inf
    for square_point in square_new:
        dist = np.max(np.abs(square_point))
        if dist < min_dist:
            min_dist = dist
    return min_dist


In [17]:
# define a square with 4 points
sq_pts = np.array([(0,0), (0,1), (1,1), (1,0)])
# define Linf from a square and a point
Linf_dist = Linf(sq_pts, [3,3])
print("Linf distance from square to point is: ", Linf_dist)


Linf distance from square to point is:  2


In [4]:
np.subtract([1,2,3], [1,2,3])

array([0, 0, 0])